In [ ]:
!pip install selenium
!pip install requests
!pip install fake_useragent
import cv2
import math
import numpy as np
from matplotlib import pyplot as plt
from __future__ import print_function
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from scipy import ndimage
from scipy.spatial import distance
from selenium import webdriver
from selenium.webdriver.support.select import Select
import time
import pandas as pd
import requests
import re
from fake_useragent import UserAgent
from bs4 import BeautifulSoup as bs


img = cv2.imread("＊＊＊＊＊＊＊＊＊", 1)
A4Height,A4Width=29.7,21
height = img.shape[0]
width = img.shape[1]
img1 = cv2.resize(img , (int(width*0.5), int(height*0.5)))

def Hieght_foot():
    Img = cv2.imread("＊＊＊＊＊＊＊＊＊＊＊＊＊", 1)
    A4Height, A4Width = 29.7, 21
    height = Img.shape[0]
    width = Img.shape[1]
    img1 = cv2.resize(Img, (int(width*0.5), int(height*0.5)))
    shapeHih, shapeWid = img1.shape[0], img1.shape[1]
    print(shapeHih, shapeWid)
    img_hsv = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)

    # hsv1
    hsv_min = np.array([0, 0, 0])
    hsv_max = np.array([10, 120, 200])
    hsv1 = cv2.inRange(img_hsv, hsv_min, hsv_max)
    #cv2.imshow('hsv1', hsv1)
    cv2.waitKey()

    # hsv2
    hsv_min = np.array([30, 0, 20])
    hsv_max = np.array([255, 255, 255])
    hsv2 = cv2.inRange(img_hsv, hsv_min, hsv_max)
    #cv2.imshow('hsv2', hsv2)
    cv2.waitKey()

    # mask
    mask = hsv1 + hsv2
    #cv2.imshow('mask', mask)
    cv2.waitKey()


    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    areas = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > height*width/30:
            epsilon = 0.03*cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,epsilon,True)
            hull = cv2.convexHull(approx)
            areas.append(hull)
            img2=cv2.drawContours(img1,areas,-1,(250,0,0),2) #print('area',area)
    for area in areas:
        if len(area) == 2:
            pts1 = np.float32(area)
            pts2 = np.float32([[0,0],[0,height],[width,height],[width,0]])
            m = cv2.getPerspectiveTransform(pts1,pts2)
            dst = cv2.warpPerspective(img2,m,(width,height))
                #cv2.imshow('img2',img2)



    #面積の最大値を取得
    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask)
    idx = np.argmax(stats[1:, 4])+1
    e = stats[idx, 0]
    f = stats[idx, 1]
    g = stats[idx, 2]
    h = stats[idx, 3]

    rect = cv2.rectangle(img1, (e, f), (e+g, f+h), (0, 0,255), 2)


    print("e",e,"f",f,"g",g,"h",h)
    dd=cv2.line(img1,(e,h),(e+g,h),(0, 255, 0), thickness=2,lineType=cv2.LINE_AA)

    aca=np.array([e,h])
    bcb=np.array([e+g,h])
    df=aca- bcb
    Hieght_long=np.linalg.norm(df)*A4Height/shapeWid
    Hieght_longs=math.floor(Hieght_long)#25㎝
    print(Hieght_longs)
    return Hieght_longs 

    cv2.imshow('img1', img1)# 結果表示
 
   
def get_cout_img(img):
    height = img.shape[0]
    width = img.shape[1]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret,thrsh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    contours, hierarchy = cv2.findContours(thrsh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    areas = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > height*width/30:
            #係数は0.01-0.03辺り
            epsilon = 0.03*cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,epsilon,True)
            hull = cv2.convexHull(approx)
            areas.append(hull)
            img=cv2.drawContours(img,areas,-1,(0,255,0),3)
    for area in areas:
        print('area',area)
        if len(area) == 4:
            pts1 = np.float32(area)
            pts2 = np.float32([[0,0],[0,height],[width,height],[width,0]])
            m = cv2.getPerspectiveTransform(pts1,pts2)
            dst = cv2.warpPerspective(img,m,(width,height))
            return dst
    print('failed to get rect')
    return img


class PointList():
    def __init__(self, npoints):
        self.npoints = npoints
        self.ptlist = np.empty((npoints, 2), dtype=int)#2次元配列
        self.pos = 0
        
    
    def add(self, x, y):
        if self.pos < self.npoints:
            f=self.ptlist[ self.pos,:] = [x, y]
            print("f",f)
            self.pos += 1
            return True
        return False
 
    
    

def onMouse(event, x, y, flag, params):
    wname, img, ptlist = params
    if event == cv2.EVENT_MOUSEMOVE:  # マウスが移動したときにx線とy線を更新する
        img2 = np.copy(img)
        h, w = img2.shape[0], img2.shape[1] #縦、横
        cv2.line(img2, (x, 0), (x, h - 1), (255, 0, 0))
        cv2.line(img2, (0, y), (w - 1, y), (255, 0, 0))
        cv2.imshow(wname, img2)

    if event == cv2.EVENT_LBUTTONDOWN:  # レフトボタンをクリックしたとき、ptlist配列にx,y座標を格納する
        if ptlist.add(x, y):
            print('[%d] ( %d, %d )' % (ptlist.pos - 1, x, y))
            cv2.circle(img1, (x, y), 3, (0, 255, 0), 3)
            cv2.imshow(wname, img1)
            shapeHih,shapeWid=img1.shape[0],img1.shape[1]
            print(shapeHih,shapeWid)
        else:
            print('All points have selected.  Press ESC-key.')
        if(ptlist.pos == ptlist.npoints):
            print("ptlist.ptlist",ptlist.ptlist)
            xx=cv2.line(img, (ptlist.ptlist[0][0], ptlist.ptlist[0][1]),
                     (ptlist.ptlist[1][0], ptlist.ptlist[1][1]), (0, 255, 0), 3)
            
            aa=np.array([ptlist.ptlist[0][0]])
            bb=np.array([ptlist.ptlist[0][1]])
            cc=np.array([ptlist.ptlist[1][0]])
            dd=np.array([ptlist.ptlist[1][1]]) 
            for a in aa:
                print("aa",a)
                for b in bb:
                    for c in cc:
                        for d in dd:
                            aca=np.array([a,b])
                            bcb=np.array([c,d])
                            df=bcb - aca
                            width_foot=np.linalg.norm(df)*A4Width/shapeWid
                            print(int(width_foot))
                            
                            
class Merukari:
    ua=UserAgent()
    useragent=ua.random
    url="https://www.mercari.com/jp/search/?sort_order=&keyword=%E3%83%91%E3%83%A9%E3%83%96%E3%83%BC%E3%83%84&category_root=&brand_name=&brand_id=&size_group=2&price_min=&price_max=&item_condition_id%5B1%5D=1&item_condition_id%5B2%5D=1&item_condition_id%5B3%5D=1&status_on_sale=1"
    browesr = webdriver.Chrome()
    browesr.get(url)
    url2=Hieght_foot()
    
    try:
        if url2==25:
            url2=browesr.find_element_by_xpath('/html/body/div[1]/main/div[1]/div[1]/div[2]').click()
            url3=browesr.find_element_by_xpath('/html/body/div[1]/main/div[2]/form/div[2]/div[4]/div[2]/div[2]/div[4]/label').click()
            url4=browesr.find_element_by_xpath('/html/body/div[1]/main/div[2]/form/div[2]/div[10]/button').click()

    except NoSuchElementException:
        print("画面表示には触らない！！")
    
    else:
        pass

    def get_html(self, url):
        
        
        headers={"User-Agent":self.useragent}
        res=requests.get(url,headers=headers)
        return res
    
    def get_items_df(self):
        res=self.get_html(self.url)
        soup=bs(res.content,"html.parser")
        items=soup.findAll(class_="items-box")

        output=[]        
        for item in items:
            output.append({
                "title":item.find(class_="items-box-name font-2").text,
                "url":"https://www.mercari.com"+item.find("a").get("href"),
                "picture":item.find("img").get("data-src"),
                "price":item.find(class_="items-box-price font-5").text  
            })
        items_df=pd.DataFrame(output)#上記の4種類をitems_df
        return items_df

    import time

    def detail(self, detail_sp):
        return{"title":detail_sp.find(class_="item-name").text,
         "url":("https://www.mercari.com/jp/items/"+ detail_sp.find(class_="item-action-text").get("data-key")).replace("id=",""),
         "place": detail_sp.find(class_="item-price bold").text,
         "pictures":"|".join([img.find("img").get("data-src") for img in detail_sp.findAll(class_="owl-item-inner")]),
         "Description":detail_sp.find(class_="item-description-inner").text,
         "spec_table": self.spec_table(detail_sp)
    }        

    def spec_table(self, detail_sp):
        spec_list = []
        ths = detail_sp.find(class_="item-detail-table").findAll("th")
        tds = detail_sp.find(class_="item-detail-table").findAll("td")
        for th, td in zip(ths, tds):
            _td = re.sub(r"\n+", "\n", td.text.strip())
            if td.find("a"):
                _url = td.find("a").get("href")
                if re.search(r"u/\d+", _url):
                    _td = _td + "\n" + _url

            spec_list.append("|".join([th.text, _td]))
        return "\t".join(spec_list)

    def detail_sp(self):
        results = []
        shoes_url=[]

        items_df = self.get_items_df()
        
        for idx, detail_url in enumerate(items_df.url.head(2)):
            detail_res = self.get_html(detail_url)
            detail_sp = bs(detail_res.content, "html.parser")
            results.append(detail_sp)
            shoes_url.append(detail_url)
            print(f"now {idx} of numbers" )
            print(detail_res)#レスポンスが200になっているかの表示
            print(detail_url)#URL表示
            time.sleep(10)
   
        last_result = []
        for detail_sp in results:
            detail_result = self.detail(detail_sp)
            last_result.append(detail_result)
        detail_df = pd.DataFrame(last_result)

        for c in detail_df.columns:
            detail_df[c] = detail_df[c].str.replace(",", "")
        ddd=detail_df.to_csv("mercari.csv",index=False,encoding='utf_8_sig')
      
        
        TOKEN='＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊'
        API_URL='https://notify-api.line.me/api/notify'
        send_contents='\n'.join(shoes_url)

        TOKEN_dic={'Authorization':'Bearer'+' '+TOKEN}
        send_dic={'message':send_contents}
        print(TOKEN_dic)
        print(send_dic)
        requests.post(API_URL,headers=TOKEN_dic,data=send_dic)


if __name__ == '__main__':
    wname = "MouseEven"
    cv2.namedWindow(wname)
    npoints = 2
    ptlist = PointList(npoints)
    img1 = get_cout_img(img1)
    cv2.setMouseCallback(wname,onMouse, [wname, img1,ptlist])
    cv2.waitKey()
    cv2.destroyAllWindows()
    
    meru=Merukari()#Merukari
    meru.detail_sp()